# Funciones avanzadas de preprocesado
Técnicas que utilizan un modelo ML del lenguaje para extraer información morfologica/semántica del texto

In [1]:
import spacy
import pandas as pd
from spacy import displacy

import es_core_news_md
nlp = es_core_news_md.load()

In [2]:
nlp.component_names

['tok2vec',
 'morphologizer',
 'parser',
 'senter',
 'attribute_ruler',
 'lemmatizer',
 'ner']

## Part of Speech

In [3]:
doc = nlp("el perro de Juan se comió mi bocadillo")

pd.DataFrame([(t.text, t.pos_, t.morph) for t in doc], columns=['text', 'POS', 'morph'])

,text,POS,morph
0,el,DET,"(Definite=Def, Gender=Masc, Number=Sing, PronT..."
1,perro,NOUN,"(Gender=Masc, Number=Sing)"
2,de,ADP,()
3,Juan,PROPN,()
4,se,PRON,"(Case=Dat, Person=3, PrepCase=Npr, PronType=Pr..."
5,comió,VERB,"(Mood=Ind, Number=Sing, Person=3, Tense=Past, ..."
6,mi,DET,"(Number=Sing, Number[psor]=Sing, Person=1, Pos..."
7,bocadillo,NOUN,"(Gender=Masc, Number=Sing)"


## Named Entity Recognition

In [4]:
doc = nlp("El gran escritor Miguel de Cervantes nació en Alcalá de Henares")

In [5]:
print([(ent.text, ent.label_) for ent in doc.ents])

[('Miguel de Cervantes', 'PER'), ('Alcalá de Henares', 'LOC')]


Entidades en el modelo `ES`

In [6]:
nlp.meta["labels"]["ner"]

['LOC', 'MISC', 'ORG', 'PER']

Entidades en el modelo `EN`

In [7]:
nlp_en = spacy.load("en_core_web_md")
nlp_en.meta["labels"]["ner"]

['CARDINAL',
 'DATE',
 'EVENT',
 'FAC',
 'GPE',
 'LANGUAGE',
 'LAW',
 'LOC',
 'MONEY',
 'NORP',
 'ORDINAL',
 'ORG',
 'PERCENT',
 'PERSON',
 'PRODUCT',
 'QUANTITY',
 'TIME',
 'WORK_OF_ART']

In [8]:
with open('articulo.txt', 'r', encoding='utf-8') as f:
    texto = f.read()
texto

'La Policía Nacional, en colaboración con la policía marroquí, ha desarticulado una "importante y peligrosa" célula del Estado Islámico (ISIS, en sus siglas en inglés) que pretendía impulsar atentados yihadistas en España y en otros países de Europa: "Tenía como objetivo materializar la estrategia de la organización terrorista en occidente", aseguran fuentes de la investigación, que aseguran que estaban en contacto con yihadistas ubicados en Siria. Los agentes de la Comisaría General de Información (CGI) y los de la Dirección General de Vigilancia del Territorio del Reino de Marruecos (DGST) han detenido en total a cinco personas, dos en España, en las localidades de Lorca (Murcia) y Abadiño (Bizkaia); y otras tres en Marruecos, que integraban la red de la organización terrorista.'

In [9]:
doc_articulo = nlp(texto)
displacy.render(doc_articulo, style='ent', jupyter=True)

In [10]:
entidades = [e for e in doc.ents]
entidades

[Miguel de Cervantes, Alcalá de Henares]

In [11]:
type(entidades[0])

spacy.tokens.span.Span

In [12]:
entidades[0].start

3

In [13]:
entidades[0].end

6

In [14]:
doc[3:6]

Miguel de Cervantes

In [15]:
entidades[0].label_

'PER'

In [16]:
datos = map(lambda t: {'token': t.orth_,
                       'POS': t.pos_,
                       'ent_iob': t.ent_iob_,
                       'ent_type': t.ent_type_
                      }, doc)

pd.DataFrame(datos)

,token,POS,ent_iob,ent_type
0,El,DET,O,
1,gran,ADJ,O,
2,escritor,NOUN,O,
3,Miguel,PROPN,B,PER
4,de,ADP,I,PER
5,Cervantes,PROPN,I,PER
6,nació,VERB,O,
7,en,ADP,O,
8,Alcalá,PROPN,B,LOC
9,de,ADP,I,LOC


Podemos usar la detección de entidades de tipo `PER` para enmascarar nombres propios en un texto

In [17]:
texto = "Al detenido Alejandro García se le acusa de matar a Juan Pérez"

In [18]:
doc = nlp(texto)
datos = map(lambda t: {'token': t.orth_,
                       'POS': t.pos_,
                       'ent_iob': t.ent_iob_,
                       'ent_type': t.ent_type_
                      }, doc)

pd.DataFrame(datos)

,token,POS,ent_iob,ent_type
0,Al,ADP,O,
1,detenido,ADJ,O,
2,Alejandro,PROPN,B,PER
3,García,PROPN,I,PER
4,se,PRON,O,
5,le,PRON,O,
6,acusa,VERB,O,
7,de,ADP,O,
8,matar,VERB,O,
9,a,ADP,O,


In [19]:
[t.text if not t.ent_type_=='PER' else '_PERSONA_' for t in doc]

['Al',
 'detenido',
 '_PERSONA_',
 '_PERSONA_',
 'se',
 'le',
 'acusa',
 'de',
 'matar',
 'a',
 '_PERSONA_',
 '_PERSONA_']

In [20]:
nlp.add_pipe("merge_entities")

<function spacy.pipeline.functions.merge_entities(doc: spacy.tokens.doc.Doc)>

In [21]:
nlp.component_names

['tok2vec',
 'morphologizer',
 'parser',
 'senter',
 'attribute_ruler',
 'lemmatizer',
 'ner',
 'merge_entities']

In [22]:
doc = nlp(texto)
datos = map(lambda t: {'token': t.orth_,
                       'POS': t.pos_,
                       'ent_iob': t.ent_iob_,
                       'ent_type': t.ent_type_
                      }, doc)

pd.DataFrame(datos)

,token,POS,ent_iob,ent_type
0,Al,ADP,O,
1,detenido,ADJ,O,
2,Alejandro García,PROPN,B,PER
3,se,PRON,O,
4,le,PRON,O,
5,acusa,VERB,O,
6,de,ADP,O,
7,matar,VERB,O,
8,a,ADP,O,
9,Juan Pérez,PROPN,B,PER


In [23]:
[t.text if not t.ent_type_=='PER' else '_PERSONA_' for t in doc]

['Al',
 'detenido',
 '_PERSONA_',
 'se',
 'le',
 'acusa',
 'de',
 'matar',
 'a',
 '_PERSONA_']

### Creación de nuevas entidades en spaCy
Podemos crear entidades nuevas con el componente `EntityRuler` del pipeline (https://spacy.io/api/entityruler)

In [24]:
#Definimos "escritor" como profesión
ruler = nlp.add_pipe("entity_ruler")
patterns = [{"label": "PROFESION", "pattern": [{"LEMMA": "escritor"}]}]
ruler.add_patterns(patterns)

doc = nlp("El gran escritor Miguel de Cervantes nació en Alcalá de Henares")
displacy.render(doc, style='ent', jupyter=True)

In [25]:
datos = map(lambda t: {'token': t.orth_,
                       'POS': t.pos_,
                       'ent_iob': t.ent_iob_,
                       'ent_type': t.ent_type_
                      }, doc)

pd.DataFrame(datos)

,token,POS,ent_iob,ent_type
0,El,DET,O,
1,gran,ADJ,O,
2,escritor,NOUN,B,PROFESION
3,Miguel de Cervantes,PROPN,B,PER
4,nació,VERB,O,
5,en,ADP,O,
6,Alcalá de Henares,PROPN,B,LOC


Podemos añadir nuevas profesiones.  
### Ejercicio
Añade "matador de toros" como profesión y comprueba si funciona con la frase siguiente:

In [33]:
patterns = [{"label": "PROFESION", "pattern": [{"LOWER": "matador"}, {"LOWER": "de"}, {"LOWER": "toros"}]}]
ruler.add_patterns(patterns)

doc = nlp("El padre de Miguel Bosé era el matador de toros Luis Miguel Dominguín")
print([(ent.text, ent.label_) for ent in doc.ents])

[('Miguel Bosé', 'PER'), ('matador de toros', 'PROFESION'), ('Luis Miguel Dominguín', 'PER')]


In [34]:
datos = map(lambda t: {'token': t.orth_,
                       'POS': t.pos_,
                       'ent_iob': t.ent_iob_,
                       'ent_type': t.ent_type_
                      }, doc)

pd.DataFrame(datos)

,token,POS,ent_iob,ent_type
0,El,DET,O,
1,padre,NOUN,O,
2,de,ADP,O,
3,Miguel Bosé,PROPN,B,PER
4,era,AUX,O,
5,el,DET,O,
6,matador,NOUN,B,PROFESION
7,de,ADP,I,PROFESION
8,toros,NOUN,I,PROFESION
9,Luis Miguel Dominguín,PROPN,B,PER


Como el proceso `entity_ruler` está detrás del `merge_entities` en el *pipeline* de `spaCy` no une las profesiones

In [35]:
nlp.component_names

['tok2vec',
 'morphologizer',
 'parser',
 'senter',
 'attribute_ruler',
 'lemmatizer',
 'ner',
 'merge_entities',
 'entity_ruler']

In [36]:
nlp.remove_pipe('merge_entities')

('merge_entities',
 <function spacy.pipeline.functions.merge_entities(doc: spacy.tokens.doc.Doc)>)

In [37]:
doc = nlp("Luis Miguel era matador de toros y Cervantes era escritor")

datos = map(lambda t: {'token': t.orth_,
                       'POS': t.pos_,
                       'ent_iob': t.ent_iob_,
                       'ent_type': t.ent_type_
                      }, doc)

pd.DataFrame(datos)

,token,POS,ent_iob,ent_type
0,Luis,PROPN,B,PER
1,Miguel,PROPN,I,PER
2,era,AUX,O,
3,matador,NOUN,B,PROFESION
4,de,ADP,I,PROFESION
5,toros,NOUN,I,PROFESION
6,y,CCONJ,O,
7,Cervantes,PROPN,B,PER
8,era,AUX,O,
9,escritor,NOUN,B,PROFESION


In [38]:
nlp.add_pipe("merge_entities")
nlp.component_names

['tok2vec',
 'morphologizer',
 'parser',
 'senter',
 'attribute_ruler',
 'lemmatizer',
 'ner',
 'entity_ruler',
 'merge_entities']

In [39]:
doc = nlp("Luis Miguel era matador de toros y Cervantes era escritor")

datos = map(lambda t: {'token': t.orth_,
                       'POS': t.pos_,
                       'ent_iob': t.ent_iob_,
                       'ent_type': t.ent_type_
                      }, doc)

pd.DataFrame(datos)

,token,POS,ent_iob,ent_type
0,Luis Miguel,PROPN,B,PER
1,era,AUX,O,
2,matador de toros,NOUN,B,PROFESION
3,y,CCONJ,O,
4,Cervantes,PROPN,B,PER
5,era,AUX,O,
6,escritor,NOUN,B,PROFESION


In [40]:
patterns = [{"label": "ANIMAL", "pattern": [{"LEMMA": "gato"},{"LEMMA": "montés", 'OP': '?'}]},
           {"label": "ANIMAL", "pattern": [{"LEMMA": "perro"}]}]
ruler.add_patterns(patterns)

doc = nlp("Los gatos y los perros son animales de compañía.")
print([(ent.text, ent.label_) for ent in doc.ents])

[('gatos', 'ANIMAL'), ('perros', 'ANIMAL')]


In [41]:
datos = map(lambda t: {'token': t.orth_,
                       'POS': t.pos_,
                       'lemma': t.lemma_,
                       'ent_iob': t.ent_iob_,
                       'ent_type': t.ent_type_
                      }, doc)

pd.DataFrame(datos)

,token,POS,lemma,ent_iob,ent_type
0,Los,DET,el,O,
1,gatos,NOUN,gato,B,ANIMAL
2,y,CCONJ,y,O,
3,los,DET,el,O,
4,perros,NOUN,perro,B,ANIMAL
5,son,AUX,ser,O,
6,animales,NOUN,animal,O,
7,de,ADP,de,O,
8,compañía,NOUN,compañía,O,
9,.,PUNCT,.,O,


In [42]:
doc = nlp("El gato montés es más grande que los gatos callejeros")

datos = map(lambda t: {'token': t.orth_,
                       'POS': t.pos_,
                       'lemma': t.lemma_,
                       'ent_iob': t.ent_iob_,
                       'ent_type': t.ent_type_
                      }, doc)

pd.DataFrame(datos)

,token,POS,lemma,ent_iob,ent_type
0,El,DET,el,O,
1,gato montés,NOUN,gato montés,B,ANIMAL
2,es,AUX,ser,O,
3,más,ADV,más,O,
4,grande,ADJ,grande,O,
5,que,SCONJ,que,O,
6,los,DET,el,O,
7,gatos,NOUN,gato,B,ANIMAL
8,callejeros,ADJ,callejero,O,


In [43]:
nlp.component_names

['tok2vec',
 'morphologizer',
 'parser',
 'senter',
 'attribute_ruler',
 'lemmatizer',
 'ner',
 'entity_ruler',
 'merge_entities']

In [44]:
print([(ent.text, ent.label_) for ent in doc.ents])

[('gato montés', 'ANIMAL'), ('gatos', 'ANIMAL')]


In [46]:
doc = nlp("El escritor Miguel de Cervantes tenía un gato llamado Bigotes")
print([(ent.text, ent.label_) for ent in doc.ents])

[('escritor', 'PROFESION'), ('Miguel de Cervantes', 'PER'), ('gato', 'ANIMAL'), ('Bigotes', 'MISC')]


## Entity linking
Usamos la API de DBPedia Spotlight (https://www.dbpedia-spotlight.org/api)

In [48]:
import requests
from IPython.display import display, HTML
# An API Error Exception
class APIError(Exception):
    def __init__(self, status):
            self.status = status
    def __str__(self):
            return "APIError: status={}".format(self.status)
      
# Base URL for Spotlight API
base_url = "http://api.dbpedia-spotlight.org/en/annotate"
# Parameters 
# 'text' - text to be annotated 
# 'confidence' -   confidence score for linking
text = "The Space Shuttle was a partially reusable low Earth orbital \
spacecraft system operated from April 12, 1981, to July 21, 2011, by \
the National Aeronautics and Space Administration in the United States. \
Launched from the Kennedy Space Center in Florida, five Space Shuttle \
orbiter vehicles flew on a total of 135 missions during 30 years."

params = {"text": text, "confidence": 0.6}
# Response content type
headers = {'accept': 'text/html'}
# GET Request
res = requests.get(base_url, params=params, headers=headers)
if res.status_code != 200:
    # Something went wrong
    raise APIError(res.status_code)
# Display the result as HTML in Jupyter Notebook
display(HTML(res.text))

APIError: APIError: status=503

In [ ]:
res.headers

In [ ]:
print(res.text)

Podemos hacer la petición en formato JSON

In [ ]:
headers = {'accept': 'application/json'}
# GET Request
res = requests.get(base_url, params=params, headers=headers)
if res.status_code != 200:
    # Something went wrong
    raise APIError(res.status_code)
# Display the result as HTML in Jupyter Notebook
respuesta = res.json()
respuesta

La respuesta JSON se puede iterar como un diccionario de claves-valores:

In [ ]:
for key, value in respuesta.items():
    print(f"{key} : {value}\n")

Además podemos acceder directamente a una clave o a una clave anidada:

In [ ]:
respuesta['@text']

In [ ]:
type(respuesta['Resources'])

In [ ]:
respuesta['Resources'][0]

In [ ]:
respuesta['Resources'][0]['@URI']

In [ ]:
for r in respuesta['Resources']:
    print(r['@URI'])

In [ ]:
    
# Base URL for Spotlight API
base_url = "http://api.dbpedia-spotlight.org/es/annotate"
# Parameters 
# 'text' - text to be annotated 
# 'confidence' -   confidence score for linking
with open('articulo.txt', 'r') as f:
    texto = f.read()
    
params = {"text": texto, "confidence": 0.5}
# Response content type
headers = {'accept': 'application/json'}
# GET Request
res = requests.get(base_url, params=params, headers=headers)
if res.status_code != 200:
    # Something went wrong
    raise APIError(res.status_code)
# Display the result as HTML in Jupyter Notebook
respuesta = res.json()
respuesta